In [55]:
import cv2
import numpy as np
import pickle

class ParkingSpaceDetector:
    PARKING_SPACE_WIDTH = 120
    PARKING_SPACE_HEIGHT = 55

    def __init__(self, image_file='img.png', video_file='carPark.mp4'):
        self.parking_positions = self.load_parking_positions()
        self.image_file = image_file
        self.video_file = video_file

        # 创建一个命名窗口并设置鼠标回调函数
        cv2.namedWindow("Parking Space Detection")
        cv2.setMouseCallback("Parking Space Detection", self.on_mouse_click)

    def load_parking_positions(self, file_name='carParkPos'):
        try:
            with open(file_name, 'rb') as file:
                return pickle.load(file)
        except FileNotFoundError:
            return []

    def on_mouse_click(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.parking_positions.append((x, y))
        elif event == cv2.EVENT_RBUTTONDOWN:
            for i, pos in reversed(list(enumerate(self.parking_positions))):
                pos_x, pos_y = pos
                if pos_x < x < pos_x + self.PARKING_SPACE_WIDTH and pos_y < y < pos_y + self.PARKING_SPACE_HEIGHT:
                    self.parking_positions.pop(i)

        # 保存更新后的位置到文件
        with open('carParkPos', 'wb') as file:
            pickle.dump(self.parking_positions, file)

    def display_parking_space_image(self):
        while True:
            img = cv2.imread(self.image_file)  # 加载图像
            for pos in self.parking_positions:
                cv2.rectangle(img, pos, (pos[0] + self.PARKING_SPACE_WIDTH, pos[1] + self.PARKING_SPACE_HEIGHT), (0, 0, 255), 2)

            # 在左上角显示文本
            text_instructions = "left click: to mark parking space, right click: to erase"
            cv2.putText(img, text_instructions, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

            cv2.imshow("Parking Space Detection", img)

            key = cv2.waitKey(1) & 0xFF
            if key == 27 or cv2.getWindowProperty("Parking Space Detection", cv2.WND_PROP_VISIBLE) < 1:
                break

        cv2.destroyAllWindows()

    def process_parking_frame(self, img_processed, img):
        free_space_count = 0
        for pos in self.parking_positions:
            pos_x, pos_y = pos
            crop = img_processed[pos_y:pos_y + self.PARKING_SPACE_HEIGHT, pos_x:pos_x + self.PARKING_SPACE_WIDTH]
            non_zero_count = cv2.countNonZero(crop)

            if non_zero_count < 100:
                free_space_count += 1
                rectangle_color = (0, 255, 0)
            else:
                rectangle_color = (0, 0, 255)

            cv2.rectangle(img, (pos_x, pos_y), (pos_x + self.PARKING_SPACE_WIDTH, pos_y + self.PARKING_SPACE_HEIGHT), rectangle_color, 2)

        # 在图像上显示可用空间计数
        text = f'Free packing space: {free_space_count}/{len(self.parking_positions)}'
        text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0]

        # 计算矩形的坐标
        text_x = 50
        text_y = 60
        rectangle_x1 = text_x - 5
        rectangle_y1 = text_y - text_size[1] - 5
        rectangle_x2 = text_x + text_size[0] + 5
        rectangle_y2 = text_y + 5

        # 绘制矩形
        cv2.rectangle(img, (rectangle_x1, rectangle_y1), (rectangle_x2, rectangle_y2), (255, 0, 0), -1) 

        # 绘制文本
        cv2.putText(img, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
    # 对视频帧进行处理
    def process_video(self):
        cap = cv2.VideoCapture(self.video_file)
        
        while True:
            ret, img = cap.read()
            if not ret:
                break
            # 将图像转换为灰度图
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # 使用高斯模糊去噪声，减少细节
            blur = cv2.GaussianBlur(gray, (3, 3), 1)
            # 自适应阈值二值化处理，突出显示停车位区域
            threshold = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 25, 16)
            # 使用中值模糊进一步去噪声
            blur = cv2.medianBlur(threshold, 5)
            # 创建形态学操作的结构元素
            kernel = np.ones((3, 3), np.uint8)
            # 膨胀操作，增强停车位区域的可见性
            dilate = cv2.dilate(blur, kernel, iterations=1)

            # 处理帧以检测停车空间
            self.process_parking_frame(dilate, img)
            # 显示处理后的图像

            cv2.imshow("Parking Space Video", img)

            # 检查是否按下Esc键或关闭窗口
            key = cv2.waitKey(1) & 0xFF
            if key == 27 or cv2.getWindowProperty("Parking Space Video", cv2.WND_PROP_VISIBLE) < 1:
                break

        cap.release()
        cv2.destroyAllWindows()

def main():
    parking_space_detector = ParkingSpaceDetector()
    parking_space_detector.display_parking_space_image()
    parking_space_detector.process_video()

if __name__ == "__main__":
    main()
